In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np

In [2]:
# -*- coding:utf-8 -*-
# @author: ZHU Feng
# @Xinxiang Meteorological Bureau

import pandas as pd 
import psycopg2

from datetime import datetime, timedelta

# import sys
# sys.path.insert(1, '/home/zhuf/CodingSpace/pyMicaps')
from pymicaps.utils import latest_init_time
from pymicaps import CIMISSQuery
import json


def daily_max_min_tmp(target_dates, day_span='20', time_zone=8, admin_code='410700'):
    """
     获取一系列日期内的24小时最高最低气温
    :param target_dates: pd.Timestamp类型的日期构成的列表
    :param day_span:
    :param time_zone:
    :param admin_code:
    :return:
    """

    client = CIMISSQuery()
    interface_id = "getSurfEleInRegionByTime"

    data = {}
    for date in target_dates:
        # target_date = datetime.strptime('%s %s:00' % (target_date, day_span), '%Y-%m-%d %H:00')
        target_date = date + timedelta(hours=int(day_span))
        target_date_index = pd.Timestamp(target_date.strftime('%Y-%m-%d %H:00+08'))

        if target_date > datetime.now():
            data[date] = pd.DataFrame(columns=['station_id', 'max_tmp', 'min_tmp'])
            continue

        times = (target_date - timedelta(hours=int(time_zone))).strftime('%Y%m%d%H0000')
        # 参数列表
        params = {'dataCode': 'SURF_CHN_MUL_HOR_N',  # 资料代码（单个）
                  'times': times,  # 时间
                  'adminCodes': admin_code,  # 国内行政编码
                  'elements': 'Station_Id_C,TEM_Max_24h,TEM_Min_24h',  # 要素字段代码；统计接口分组字段
                  }

        result = client.callAPI_to_serializedStr(interface_id, params, dataFormat='json')
        t = json.loads(result)
        if t.get('returnCode', None) != '0':  # 没有数据或查询失败 返回空的dataframe
            df = pd.DataFrame(columns=['station_id', 'max_tmp', 'min_tmp'])
        else:
            df = pd.DataFrame.from_records(t['DS'])

            df.rename(columns={'Station_Id_C': 'station_id', 'TEM_Max_24h': 'max_tmp', 'TEM_Min_24h': 'min_tmp'},
                      inplace=True)
            df[['max_tmp', 'min_tmp']] = df[['max_tmp', 'min_tmp']].astype(float)

            df.set_index('station_id', inplace=True)

            df.columns = pd.MultiIndex.from_product([[target_date_index], ['max_tmp', 'min_tmp']],
                                                    names=['target_date', 'data'])
        
        # CIMISS有时候会把获嘉站重复查询出来，不知何故，需要去除重复，否则其结果被用于计算误差时候会出错
        # df.drop_duplicates(inplace=True)
        df = df[~df.index.duplicated()]

        data[date] = df

    client.destroy()

    return data


def tmp_forecast_analysis(model_name, element_type, level_type, day_span='20', infer_date=None, before_days=10, after_days=7,
                          city='新乡市', method_name='IDW'):
    """

    :param model_name:
    :param day_span:
    :param infer_date: 参考日期,默认是今天
    :param before_days:
    :param after_days:
    :param city:
    :param method_name:
    :return:
    """
    if infer_date is None:
        infer_date = pd.Timestamp.today().date()
    else:
        infer_date = pd.Timestamp(infer_date).date()

    dates = pd.date_range(infer_date - pd.Timedelta(days=before_days), periods=before_days + after_days)
    truth_dict = daily_max_min_tmp(dates, day_span)

    df_list = []
    with psycopg2.connect(host='172.18.168.11', dbname='Micaps', user='postgres', password='postgres') as conn:
        with conn.cursor() as cur:
            for date in dates:

                target_date = date.strftime('%Y-%m-%d')
                target_date_end = datetime.strptime('%s %s:00' % (target_date, day_span), '%Y-%m-%d %H:00')
                target_date_beg = target_date_end - timedelta(days=1)

                latest_init = latest_init_time('')        # 当前最临近起报时间
                init_min = target_date_end - timedelta(hours=240)    # 根据目标时间计算最早起报时间
                if init_min > latest_init:                        # 如果最早起报时间比最临近起报时间还晚,则没有数据,报错
                    raise Exception('no predict products yet')

                # 根据目标时间计算最晚起报时间,为了获取完整的24h,需要考虑目标时间的左边界与最临近起报时间的大小
                init_max = target_date_beg if target_date_beg <= latest_init else latest_init

                target_date_beg = target_date_beg.strftime('%Y-%m-%d %H:00+08')
                target_date_end = target_date_end.strftime('%Y-%m-%d %H:00+08')
                init_min = init_min.strftime('%Y-%m-%d %H:00+08')
                init_max = init_max.strftime('%Y-%m-%d %H:00+08')

                cur.execute('''SELECT station, (%s - init_time) AS forecast_period, init_time, max(value), min(value)
                               FROM extractions, stations, methods, micaps_files
                               WHERE station=stations.id and method=methods.id AND micaps_file=micaps_files.id and
                               model_name=%s AND element_type=%s AND level_type=%s AND method_name=%s AND city=%s AND
                               init_time + (forecast_period::text || ' hours')::interval >=%s AND
                               init_time + (forecast_period::text || ' hours')::interval <=%s AND
                               init_time >= %s AND init_time <= %s
                               GROUP BY station, init_time
                               ORDER BY station, init_time
                            ''', (target_date_end, model_name, element_type, level_type, method_name, city,
                                  target_date_beg, target_date_end, init_min, init_max))

                model = pd.DataFrame.from_records(cur, columns=['station_id', 'forecast_period', 'init_time',
                                                                'max_tmp', 'min_tmp'])
                model.set_index(['station_id', 'forecast_period'], inplace=True)
                model.columns = pd.MultiIndex.from_product([[pd.Timestamp(target_date_end)],
                                                            ['init_time', 'max_tmp', 'min_tmp']],
                                                           names=['target_date', 'data'])
                # 计算预报误差
                truth = truth_dict[date]
                if not truth.empty:  # 还没有实况的不计算
                    model.iloc[:, [1, 2]] = model.iloc[:, [1, 2]] - truth
                    model.rename(columns={'max_tmp': 'max_tmp_error', 'min_tmp': 'min_tmp_error'}, inplace=True)
                    # result = pd.concat([model, error], axis=1)

                df_list.append(model)

    merged = pd.concat(df_list, axis=1)
    return merged


def predict_trend(model_name, target_date, day_span='20', method_name='IDW'):
    """
    分组统计查询每个站点上不同起报时次在固定时间段内的最高最低气温
    :param model_name:
    :param target_date:
    :param day_span:
    :param method_name:
    :return:
    """
    target_date_end = datetime.strptime('%s %s:00' % (target_date, day_span), '%Y-%m-%d %H:00')
    target_date_beg = target_date_end - timedelta(days=1)

    latest_init = latest_init_time('')
    init_min = target_date_end - timedelta(hours=240)
    if init_min > latest_init:
        raise Exception('no predict products yet')
    init_max = target_date_beg if target_date_beg <= latest_init else latest_init

    target_date_beg = target_date_beg.strftime('%Y-%m-%d %H:00+08')
    target_date_end = target_date_end.strftime('%Y-%m-%d %H:00+08')
    init_min = init_min.strftime('%Y-%m-%d %H:00+08')
    init_max = init_max.strftime('%Y-%m-%d %H:00+08')

    with psycopg2.connect(host='172.18.168.11', dbname='Micaps', user='postgres', password='postgres') as conn:
        with conn.cursor() as cur:
            cur.execute('''SELECT station, (%s - init_time) as forecast_period, init_time, max(value), min(value)
                           FROM extractions, stations, methods, micaps_files
                           WHERE station=stations.id and method=methods.id and micaps_file=micaps_files.id and
                           model_name=%s and method_name=%s and city='新乡市' and
                           init_time + (forecast_period::text || ' hours')::interval >=%s and
                           init_time + (forecast_period::text || ' hours')::interval <=%s and
                           init_time >= %s and init_time <= %s
                           GROUP BY station, init_time
                           ORDER BY station, init_time
                        ''', (target_date_end, model_name, method_name, target_date_beg, target_date_end,
                              init_min, init_max))

            df = pd.DataFrame.from_records(cur, columns=['station_id', 'forecast_period', 'init_time',
                                                         'max_tmp', 'min_tmp'])
            df.set_index(['station_id', 'forecast_period'], inplace=True)
            df.columns = pd.MultiIndex.from_product([[pd.Timestamp(target_date_end)],
                                                     ['init_time', 'max_tmp', 'min_tmp']],
                                                    names=['target_date', 'data'])
    return df


def get_max_min_tmp(target_date, day_span='20', admin_code='410700', time_zone=8):
    target_date = datetime.strptime('%s %s:00' % (target_date, day_span), '%Y-%m-%d %H:00')
    target_date_index = target_date.strftime('%Y-%m-%d %H:00+08')
    times = (target_date - timedelta(hours=time_zone)).strftime('%Y%m%d%H0000')

    client = CIMISSQuery()

    interface_id = "getSurfEleInRegionByTime"

    # 参数列表
    params = {'dataCode': 'SURF_CHN_MUL_HOR_N',  # 资料代码（单个）
              'times': times,  # 时间
              'adminCodes': admin_code,  # 国内行政编码
              'elements': 'Station_Id_C,TEM_Max_24h,TEM_Min_24h',  # 要素字段代码；统计接口分组字段
              }

    result = client.callAPI_to_serializedStr(interface_id, params, dataFormat='json')

    client.destroy()

    t = json.loads(result)
    if t.get('returnCode', None) != '0':  # 没有数据或查询失败 返回空的dataframe
        df = pd.DataFrame(columns=['station_id', 'max_tmp', 'min_tmp'])
    else:
        df = pd.DataFrame.from_records(t['DS'])

        df.rename(columns={'Station_Id_C': 'station_id', 'TEM_Max_24h': 'max_tmp', 'TEM_Min_24h': 'min_tmp'},
                  inplace=True)
        df[['max_tmp', 'min_tmp']] = df[['max_tmp', 'min_tmp']].astype(float)

        df.set_index('station_id', inplace=True)

        df.columns = pd.MultiIndex.from_product([[pd.Timestamp(target_date_index)],
                                                 ['max_tmp', 'min_tmp']], names=['target_date', 'data'])

    return df


def init_time_span(target_date, day_span='20', format='%Y-%m-%d %H:00'):
    target_date_end = datetime.strptime('%s %s:00' % (target_date, day_span), '%Y-%m-%d %H:00')

    target_date_beg = target_date_end - timedelta(days=1)
    latest_init = latest_init_time('')

    init_min = target_date_end - timedelta(hours=240)
    if init_min > latest_init:
        raise Exception('no predict products')
    init_max = target_date_beg if target_date_beg <= latest_init else latest_init

    init_time = init_min
    while init_time <= init_max:
        yield init_time.strftime(format)
        init_time = init_time + timedelta(hours=12)


def validation_comparision(target_date, day_span='20'):
    model = predict_trend('ECMWF_HR', target_date, day_span)
    truth = get_max_min_tmp(target_date, day_span)

    if not truth.empty:
        model.iloc[:, [1, 2]] = model.iloc[:, [1, 2]] - truth
        model.rename(columns={'max_tmp': 'max_tmp_error', 'min_tmp': 'min_tmp_error'}, inplace=True)
        # result = pd.concat([model, error], axis=1)
    return model

In [3]:
def heatmap_plot(data, station_id, to_plot_T, model_name, fixed_cmap=True):
    """
    绘制单站2m温度的检验结果的heatmap图
    
    data: 由tmp_forecast_analysis查询获取的分析结果
    station_id: 字符型站点ID号
    to_plot_T:'max_tmp_error','min_tmp_error','max_tmp','min_tmp'
    """
    if station_id == 'average':
        df = data.mean(level='forecast_period')
    else:
        df = data.loc[station_id]
    #ddf =df.iloc[:, [i for i in range(df.shape[1]) if i%3==2]].iloc[:,:11]

    ddf = df.xs(to_plot_T, level = 'data', axis=1, drop_level=True)
    ddf.columns = ddf.columns.strftime('%m%d%H')
    ddf.columns.name = '日期'  #'target_datetime'
    
    ddf.index = ['%d h' %(td.days*24 + td.seconds/(60*60)) for td in ddf.index]
    ddf.index.name = '时效'  # 'forecast_period'
     
    colormap =cm.get_cmap("bwr")
    error_color_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]
    
    colormap =cm.get_cmap("Greens")
    T_color_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]
    
    if fixed_cmap:
        mx = 8 # ddf.max().max()
        mn = -8 # ddf.min().min()
    else:
        mx = ddf.max().max()
        mn = ddf.min().min()
    
    def error_background_color(val):
        """
        为pandas表格背景设置基于元素值的背景色
        """
        
        if np.isnan(val):
            return 'color: white; background-color: white' 

        if val<=0:
            index = int(128-128*val/mn)
        else:
            index = int(128+128*val/mx)
                       
        if index<0:
            index = 0
        elif index>255:
            index = 255

        color = error_color_palette[index]
        
        return 'background-color: %s' % color
    
    def T_background_color(col):
        colors_list = []
        span = col.max() - col.min()
        for v in col:
            if np.isnan(v):
                colors_list.append('color: white; background-color: white')
            else:
                index = int((v-col.min())/span*250)
                if index<0:
                    index = 5
                elif index>255:
                    index = 250
                colors_list.append('background-color: %s' % T_color_palette[index])
        return colors_list
    
    
    if to_plot_T in ['max_tmp_error','min_tmp_error']:
        df_heatmap = (ddf.style
                         .applymap(error_background_color)
                         .format("{:<+.1f}")
                         .set_caption("%s %s 2T_%s 过去20天预报检验"%(station_id, model_name, to_plot_T[:3])))
    else:
        df_heatmap = (ddf.style
                         .apply(T_background_color)
                         .format("{:<.1f}")
                         .set_caption("%s %s 2T_%s 未来七天预报"%(station_id, model_name, to_plot_T[:3])))
    return df_heatmap

## 说明
    - 每次打开本网页，需要点击菜单 Run -> Restart Kernel and Run All Cells，才可获取最新检验和预报结果，当然不需要重复点击，一般半天点一次，跟模式时次差不多
    - 如果修改参数设置，仍然要点击菜单 Run -> Restart Kernel and Run All Cells，才可获取新的检验和预报结果
    - 温度检验的色标是全局性的，红表示偏高，蓝色表示偏低，颜色越深表示偏差越大
    - 温度预报的色标是基于列的, 绿色程度越深表示温度越高

## <center> <font size=5> <b> 参数设置 </b> </font> </center>

In [4]:
# 待检验的模式， 其他可用模式名称'GRAPES_GFS', 'GRAPES_MESO','JAPAN_MR','BEIJING_MR','SHANGHAI_MR','GERMAN_HR'
model = 'ECMWF_HR'

# 日界,'08'或 '20', 无需设置，默认按照上午'08'，下午'20'。
# 也可以手动指定，如需指定，需要去除day_span前面的井号 #
#day_span = '08'



In [5]:
# 日界,'08'或 '20', 默认按照上午'08'，下午'20'
day_span = latest_init_time(now =datetime.now() - timedelta(hours=12))[-2:]

# 查并分析2m温度的检验结果并分析
data = tmp_forecast_analysis(model,'TMP','2M_ABOVE_GROUND', day_span, before_days=19, after_days=8)

## <center> <font size=5> <b> 2m最低温度检验 </b> </font> </center>

### 平均

In [6]:
heatmap_plot(data,'average','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,+0.2,+1.7,+0.4,+0.4,-0.3,-0.5,+2.0,+1.4,+0.5,+0.9,+0.6,+0.0,-0.4,-0.3,-1.3,-1.3,+0.5,+0.6,-0.1
36 h,+0.3,+1.6,+0.7,+0.8,-0.1,-0.8,+2.1,+1.1,+1.5,+1.0,+0.6,-0.1,-0.5,-0.7,-2.0,-1.5,+0.7,+1.1,-0.5
48 h,+0.4,+1.5,+0.6,+1.0,+0.3,-0.5,+2.0,+1.2,+1.4,+1.0,+0.6,-0.2,+1.2,-0.2,-1.5,-1.2,+0.9,+1.6,-0.5
60 h,+0.4,+1.7,+1.4,+0.7,-0.7,+0.1,+2.0,+0.9,+1.8,+0.5,+0.2,-0.3,+1.2,-0.6,-1.0,-0.6,-0.7,+1.4,-1.2
72 h,+0.5,+1.8,+1.7,+0.7,+0.4,+1.2,+1.8,+0.9,+2.0,+0.8,+0.3,-1.0,+0.7,+1.0,-2.9,-1.3,-0.2,+1.7,-1.9
84 h,+0.5,+1.9,+0.7,+0.5,+0.9,+0.3,+1.2,+0.7,+1.7,+0.6,+0.4,-1.1,+0.9,+0.9,-4.3,-3.8,-0.4,+0.9,-1.2
96 h,+1.5,+2.8,+1.7,+0.6,-0.0,-0.4,+2.9,+1.9,+1.7,+2.0,+1.0,+1.2,+1.4,+0.7,-1.2,-3.7,-1.0,+2.9,-0.5
108 h,+1.2,+2.8,+1.3,-0.3,+1.2,+1.5,+1.7,+2.1,+1.0,+2.4,+1.9,+2.3,+1.7,+0.7,-0.9,-3.6,-2.4,+2.5,+0.4
120 h,+1.1,+3.3,+2.4,+0.8,+0.5,+0.8,+3.5,+2.3,+1.7,+1.8,+2.8,+2.0,+1.2,+1.6,-0.7,-4.3,-2.3,+1.7,+0.6


### 53986

In [7]:
heatmap_plot(data,'53986','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,+0.0,+1.9,+1.6,+0.5,-0.1,-1.5,+1.7,-0.0,+0.1,-0.5,-0.0,-1.7,-0.7,-0.9,-2.0,-0.5,+0.9,+1.2,-0.5
36 h,+0.1,+1.9,+2.3,+1.0,+0.3,-1.4,+1.5,-0.7,+1.2,-0.6,+0.3,-2.2,-0.8,-1.2,-2.6,-0.9,+1.1,+1.6,-1.2
48 h,+0.1,+1.5,+2.1,+0.7,+0.6,-1.2,+1.5,-0.7,+0.7,-0.6,+0.5,-2.3,+1.0,-1.0,-1.9,-0.6,+1.4,+2.3,-1.0
60 h,+0.1,+2.1,+3.3,+0.7,-0.3,-0.4,+1.6,-1.1,+1.2,-0.5,+0.2,-2.4,+0.9,-1.3,-1.8,+0.2,-0.5,+1.9,-1.6
72 h,-0.1,+2.1,+3.4,+1.0,+0.0,+0.6,+1.5,-0.9,+1.4,-0.4,+0.5,-3.1,+0.3,+0.4,-2.9,-1.0,+0.2,+2.6,-2.7
84 h,-0.2,+2.3,+2.3,+0.1,+1.0,-0.2,+0.7,-1.1,+0.8,-0.0,+0.3,-3.1,+0.5,+0.0,-4.2,-3.4,+0.0,+1.4,-1.8
96 h,+1.2,+2.8,+3.2,+0.8,+0.1,-0.5,+2.4,+0.3,+1.2,+1.2,+0.6,-0.9,+1.1,+0.1,-1.6,-3.4,-0.9,+3.5,-1.1
108 h,+0.6,+2.8,+3.0,-0.0,+1.0,+0.9,+1.7,+0.5,+0.3,+1.4,+1.4,+0.2,+1.7,+0.1,-1.2,-3.0,-2.2,+3.1,+0.2
120 h,+0.7,+3.6,+3.6,+0.7,+0.5,+0.4,+3.1,+0.8,+0.9,+1.4,+2.3,+0.1,+0.3,+0.9,-0.9,-3.9,-2.3,+1.8,+0.3


### 53983

In [8]:
heatmap_plot(data,'53983','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,+1.1,+1.3,-0.0,+0.8,-0.6,-0.0,+3.1,+2.5,+1.3,+3.2,+2.8,+2.4,-0.0,+0.2,-0.9,-0.6,-0.1,-0.5,-0.2
36 h,+1.5,+1.4,-0.0,+1.1,-0.8,-0.6,+3.3,+2.2,+2.0,+2.8,+2.8,+2.3,-0.4,-0.0,-1.5,-0.4,-0.0,+0.3,-0.1
48 h,+1.5,+1.4,-0.1,+1.7,-0.3,+0.2,+3.3,+2.3,+2.3,+2.9,+2.5,+2.3,+1.5,+0.8,-1.1,-0.3,+0.1,+0.8,-0.1
60 h,+1.5,+1.3,+0.1,+1.3,-1.0,+0.6,+3.1,+2.1,+3.0,+2.6,+2.1,+2.4,+1.6,+0.2,+0.3,+0.0,-1.1,+0.8,-1.4
72 h,+1.9,+1.6,+0.8,+1.0,+0.9,+1.5,+2.8,+2.2,+2.9,+2.9,+2.0,+1.5,+1.0,+2.2,-3.0,+0.2,-1.1,+0.4,-1.6
84 h,+1.9,+1.7,-0.2,+1.2,+0.5,+0.7,+1.9,+2.0,+3.3,+2.1,+1.8,+1.3,+1.5,+2.4,-4.1,-2.5,-1.2,+0.2,-0.9
96 h,+2.8,+3.4,+1.0,+1.0,-0.5,-0.3,+4.0,+2.9,+2.0,+3.6,+2.9,+3.5,+1.6,+1.8,-0.1,-2.0,-1.3,+2.1,-0.1
108 h,+2.7,+3.4,+0.3,-0.5,+1.5,+2.2,+2.4,+3.2,+2.0,+4.0,+4.0,+4.9,+1.9,+1.6,-0.0,-2.8,-2.8,+1.8,+0.4
120 h,+2.4,+3.2,+1.8,+1.4,+0.4,+1.2,+4.7,+3.2,+2.9,+3.1,+4.8,+4.5,+2.0,+3.1,-0.4,-3.5,-2.5,+1.6,+0.5


### 53985

In [9]:
heatmap_plot(data,'53985','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-0.0,+1.4,-0.6,-0.5,-0.9,-0.0,+2.1,+1.6,+1.0,+1.5,+1.5,-0.6,-0.1,+0.2,-1.3,-1.3,+1.4,+1.7,-0.2
36 h,+0.1,+1.0,-0.7,-0.3,-0.8,+0.8,+2.3,+1.4,+1.8,+1.6,+1.2,-0.5,-0.6,-0.3,-1.4,-2.1,+1.7,+2.0,-1.4
48 h,+0.3,+0.6,-0.5,+0.2,+0.0,+0.3,+3.2,+1.4,+2.1,+1.3,+0.6,-0.8,+0.9,-0.2,-0.8,-1.2,+1.9,+2.3,-0.8
60 h,-0.0,+0.9,+1.0,-0.1,-1.5,+1.0,+2.7,+0.9,+2.2,+1.3,+0.4,-0.9,+1.0,+0.3,-0.3,-1.3,+0.1,+2.1,-0.9
72 h,+0.3,+1.1,+0.9,-0.3,-1.0,+2.1,+3.0,+1.2,+2.3,+0.8,+0.8,-1.3,+0.5,+1.0,-2.1,-2.4,+0.8,+2.9,-1.9
84 h,+0.2,+1.4,+0.4,-0.3,+0.1,+0.6,+2.2,+0.5,+2.2,+0.9,+0.9,-1.5,+1.0,+1.1,-3.8,-4.9,+0.2,+1.5,-1.5
96 h,+1.2,+1.9,+0.8,-0.1,-0.8,+0.3,+4.0,+2.2,+2.5,+2.1,+1.2,+0.4,+0.9,+1.0,-1.4,-4.9,-0.3,+3.6,-0.9
108 h,+0.9,+1.8,+0.6,-0.6,+0.1,+1.9,+3.0,+2.3,+1.4,+2.1,+1.6,+1.2,+0.9,+1.5,-0.8,-4.0,-1.7,+3.3,+0.4
120 h,+0.9,+2.6,+1.6,-0.2,+0.7,+1.3,+4.4,+2.4,+2.8,+1.5,+2.6,+1.3,+0.5,+1.5,-1.0,-4.9,-1.3,+1.0,+0.5


### 53988

In [10]:
heatmap_plot(data,'53988','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,+0.6,+1.6,-0.2,+0.5,+0.3,-0.9,+2.5,+2.8,+0.3,-0.0,+0.7,+0.6,-0.5,-0.6,-0.6,-0.1,+0.5,+1.9,+0.1
36 h,+0.7,+1.4,+0.2,+0.9,+0.2,-1.5,+2.7,+2.2,+1.3,+0.3,+0.9,+0.5,-0.8,-0.5,-1.4,-0.2,+1.0,+2.1,-0.8
48 h,+0.7,+1.1,+0.6,+1.0,+0.7,-1.7,+2.5,+2.1,+1.1,+0.3,+1.2,+0.6,+0.9,-0.5,-1.1,+0.2,+1.4,+2.8,-0.6
60 h,+0.4,+1.5,+2.8,+0.8,+0.2,-0.4,+2.8,+1.8,+2.1,+0.2,+0.7,+0.4,+1.4,-0.9,-0.7,+1.0,-0.3,+2.2,-1.1
72 h,+0.2,+1.8,+1.9,+0.9,+0.3,+0.9,+2.4,+1.8,+2.7,+0.1,+1.5,-0.4,+0.1,+0.7,-2.0,+0.5,+0.3,+3.3,-2.0
84 h,-0.1,+1.9,+1.2,+0.4,+1.4,-0.1,+2.0,+1.8,+1.2,+0.6,+1.0,-0.7,+0.6,+0.4,-3.7,-2.2,+0.2,+2.0,-1.2
96 h,+1.6,+2.6,+2.2,-0.1,+1.0,-1.1,+3.5,+2.5,+2.8,+1.8,+1.3,+1.6,+0.8,+0.5,+0.2,-2.2,-1.5,+3.9,-0.3
108 h,+0.8,+2.5,+1.9,+0.3,+2.0,+1.2,+2.3,+2.6,+1.4,+2.1,+1.9,+2.9,+1.6,+0.3,+0.3,-2.0,-2.5,+3.1,+0.8
120 h,+1.2,+3.0,+2.9,+0.4,+1.4,+0.4,+4.4,+2.9,+1.8,+2.4,+2.7,+2.7,+1.3,+1.1,+0.2,-3.1,-2.5,+2.3,+1.0


### 53989

In [11]:
heatmap_plot(data,'53989','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,+0.9,+2.2,+1.9,+0.8,+0.7,-0.9,+2.1,+1.5,+1.9,+2.3,+2.1,+1.4,-0.6,-0.2,-0.9,-0.4,+0.4,+1.4,+0.5
36 h,+0.8,+1.9,+2.7,+1.3,+0.9,-1.5,+1.9,+1.5,+3.3,+2.2,+2.0,+0.8,-0.5,-0.3,-1.5,-0.5,+0.3,+2.0,+0.1
48 h,+0.8,+1.5,+2.2,+1.0,+1.3,-0.7,+2.1,+1.8,+2.8,+2.9,+2.4,+1.1,+1.2,-0.1,-1.4,-0.1,+0.6,+2.5,-0.0
60 h,+1.3,+2.3,+2.7,+1.1,+0.6,-0.4,+1.9,+1.6,+3.1,+1.1,+1.8,+0.8,+1.3,-0.5,-0.7,+0.9,-1.0,+2.2,-0.7
72 h,+1.2,+2.2,+3.2,+0.9,+2.0,+0.6,+1.5,+1.5,+3.5,+1.9,+2.1,+0.0,+0.8,+1.1,-3.0,-0.0,-0.6,+2.3,-1.6
84 h,+1.2,+2.3,+2.0,+0.6,+2.5,+0.1,+1.4,+1.5,+2.8,+1.5,+2.1,-0.2,+0.7,+1.2,-4.3,-2.8,-0.5,+1.8,-0.7
96 h,+2.0,+2.7,+3.3,+1.5,+1.4,-1.1,+2.5,+2.8,+2.6,+3.5,+2.9,+2.8,+1.7,+0.7,-0.5,-2.7,-1.5,+3.9,+0.2
108 h,+1.8,+2.9,+2.9,-0.2,+2.5,+1.0,+1.5,+3.0,+2.6,+4.7,+3.5,+3.8,+2.0,+0.7,-0.4,-2.7,-2.7,+3.3,+0.7
120 h,+1.7,+3.4,+3.7,+1.0,+1.1,+0.6,+3.1,+3.4,+2.8,+3.1,+4.4,+3.6,+1.5,+1.6,-0.6,-3.1,-2.6,+2.8,+1.0


### 53994

In [12]:
heatmap_plot(data,'53994','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-0.8,+1.9,-0.0,-0.0,-0.6,+1.0,+1.3,+0.6,-0.5,-1.5,-1.2,-0.4,-0.9,-0.7,-2.4,-2.4,+1.3,+0.1,-0.6
36 h,-0.8,+1.9,+0.6,+0.4,-0.4,+0.3,+2.1,+0.2,+0.8,-1.4,-1.1,-0.2,-0.9,-1.7,-3.1,-2.7,+1.4,+0.5,-0.8
48 h,-0.7,+2.0,+0.4,+0.7,-0.0,+0.3,+1.2,+0.2,+0.4,-1.3,-1.3,-0.9,+0.5,-0.9,-2.6,-2.7,+1.4,+1.2,-0.8
60 h,-0.8,+2.1,+0.7,+0.6,-1.5,+1.3,+1.2,+0.0,+0.5,-1.7,-1.5,-1.0,+0.3,-1.4,-2.3,-2.1,-0.5,+0.9,-1.4
72 h,-0.6,+2.1,+1.1,+0.6,-0.4,+2.3,+1.3,+0.1,+1.0,-1.5,-1.9,-1.1,+0.2,+0.3,-3.6,-3.7,+0.5,+1.5,-2.5
84 h,-0.5,+2.2,-0.1,+0.2,+0.7,+1.3,+0.6,+0.1,+0.6,-1.3,-1.4,-0.7,+0.2,+0.1,-4.0,-6.0,+0.1,+0.3,-1.8
96 h,+0.4,+2.8,+1.1,+0.4,-0.8,+1.3,+2.4,+1.1,+1.1,-0.0,-0.9,+0.9,+1.7,+0.2,-2.8,-6.0,-0.4,+2.6,-1.2
108 h,+0.1,+2.9,+0.7,-0.7,+0.1,+2.5,+1.0,+1.5,-0.1,-0.1,-0.3,+1.6,+1.4,-0.0,-2.5,-5.2,-1.8,+2.1,+0.1
120 h,-0.1,+3.6,+1.5,+0.1,-0.1,+1.9,+2.6,+1.6,+0.6,-0.0,+1.0,+1.5,+0.2,+0.6,-1.4,-6.0,-1.8,+1.0,+0.2


### 53997

In [13]:
heatmap_plot(data,'53997','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,+0.2,+1.8,-0.2,+0.8,-1.0,-1.4,+1.8,+1.7,+0.7,+2.0,+0.4,-0.2,-0.6,-0.2,-1.6,-2.5,-0.1,-0.7,-0.1
36 h,+0.5,+1.7,+0.1,+1.1,-0.9,-1.7,+1.5,+1.6,+1.6,+2.3,-0.2,-0.7,-0.4,-0.4,-1.8,-2.5,-0.3,-0.1,-0.2
48 h,+0.6,+1.8,-0.0,+1.3,-0.7,-1.1,+1.7,+1.7,+2.1,+2.5,+0.3,-0.5,+1.5,+0.1,-2.0,-2.3,-0.0,+0.4,-0.6
60 h,+1.0,+1.8,+0.0,+0.8,-1.6,-0.6,+1.6,+1.6,+1.9,+0.2,-0.6,-0.4,+1.1,-0.4,-1.2,-1.4,-1.5,+0.1,-1.3
72 h,+1.0,+2.0,+0.8,+0.7,+0.2,+0.0,+1.3,+1.5,+2.5,+1.2,-0.7,-1.5,+0.9,+1.6,-3.6,-2.1,-1.1,+0.3,-2.0
84 h,+0.8,+2.0,-0.3,+0.8,+0.4,-0.3,+1.1,+1.4,+1.5,+0.6,-0.5,-1.7,+0.7,+1.2,-5.0,-4.9,-1.1,-0.2,-1.3
96 h,+1.7,+2.8,+1.0,+1.2,-0.9,-1.4,+2.5,+2.4,+1.2,+2.2,+0.6,+1.2,+1.5,+0.9,-1.5,-4.7,-1.5,+1.9,-0.5
108 h,+1.4,+2.9,+0.6,-0.7,+0.8,+0.8,+1.3,+2.8,+1.3,+3.0,+1.6,+2.5,+1.9,+0.8,-1.2,-5.0,-2.9,+1.3,+0.1
120 h,+1.4,+3.4,+1.6,+0.9,-0.5,-0.1,+3.1,+2.8,+1.6,+1.7,+2.3,+2.2,+1.6,+1.9,-1.2,-5.2,-2.9,+1.0,+0.4


### 53998

In [14]:
heatmap_plot(data,'53998','min_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-0.7,+1.3,+0.4,+0.5,+0.0,-0.6,+1.1,+0.7,-0.9,+0.3,-1.1,-1.2,+0.5,-0.6,-0.9,-2.6,-0.1,-0.2,+0.2
36 h,-0.4,+1.2,+0.5,+0.6,+0.5,-1.1,+1.5,+0.4,-0.2,+0.9,-0.9,-1.2,+0.5,-1.3,-2.6,-2.5,+0.1,+0.3,+0.2
48 h,-0.2,+1.7,+0.1,+1.2,+0.5,-0.0,+0.6,+0.4,-0.3,+0.4,-1.6,-1.2,+1.9,+0.1,-1.4,-2.5,+0.3,+0.8,+0.1
60 h,-0.2,+1.2,+0.6,+0.7,-0.8,-0.0,+0.8,+0.3,+0.2,+1.1,-1.6,-1.3,+1.9,-0.5,-1.0,-2.1,-1.1,+0.6,-1.0
72 h,+0.1,+1.5,+1.1,+0.5,+1.6,+1.2,+0.8,+0.1,+0.1,+1.0,-1.6,-2.0,+1.8,+1.1,-3.3,-1.6,-0.7,+0.2,-1.1
84 h,+0.4,+1.7,+0.1,+1.0,+0.8,-0.1,+0.1,-0.5,+1.2,+0.5,-1.3,-2.2,+2.2,+1.2,-4.9,-3.8,-1.3,+0.2,-0.6
96 h,+1.1,+3.5,+1.2,+0.5,+0.4,-0.5,+2.1,+0.7,+0.1,+2.0,-1.0,+0.1,+1.9,+0.7,-1.6,-3.3,-0.9,+2.0,+0.1
108 h,+0.9,+3.5,+0.8,+0.2,+1.5,+1.4,+0.3,+1.0,-0.6,+2.3,+1.1,+1.3,+2.2,+0.8,-1.0,-4.3,-2.4,+1.7,+0.7
120 h,+0.4,+3.2,+2.2,+1.8,+1.0,+0.8,+2.3,+1.2,+0.1,+1.6,+2.0,+0.5,+2.2,+2.2,-0.6,-4.8,-2.4,+1.7,+0.8


 ## <center> <font size=5> <b> 2m最高温度检验 </b> </font> </center>

### 平均

In [15]:
heatmap_plot(data,'average','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.4,+0.0,-0.6,+0.3,-1.5,-2.0,-1.4,-0.3,-1.1,-0.9,+0.0,-0.4,-0.6,-1.6,-0.7,-0.1,+1.9,-0.8,+2.6
36 h,-1.9,+0.2,-0.8,-0.4,-2.2,-2.2,-1.9,-0.2,-1.3,-0.9,-0.1,-0.6,+0.1,-0.8,-1.7,-0.4,+2.1,-0.6,+0.6
48 h,-2.4,+1.0,-0.7,-1.4,-1.0,-2.1,-1.9,-0.2,-1.0,-1.0,-0.3,+0.3,-1.3,-1.0,-0.3,-0.3,+2.3,-0.9,+0.7
60 h,-2.1,-0.0,-2.8,+1.0,-1.5,-2.1,-2.2,-0.4,-1.2,-0.9,-0.5,-0.5,-0.8,-2.2,+0.2,-1.5,+2.9,-0.8,+3.2
72 h,-2.2,+0.5,-1.7,-3.9,-1.0,-1.6,-2.0,-1.1,-0.9,-0.9,-0.5,-0.4,+0.0,-0.8,-0.4,-2.2,+3.3,-2.0,+3.5
84 h,-2.2,+1.3,-1.4,-2.7,+1.6,-4.4,-1.9,-1.2,-0.8,-1.4,-0.4,-0.8,-0.6,-0.4,-1.3,-4.5,+2.1,-0.5,+1.1
96 h,-2.1,+0.9,-1.4,-0.7,+0.3,-1.4,-1.9,-0.5,-0.9,-0.9,-0.7,-0.8,+0.9,-0.5,-0.1,-4.1,+0.3,-0.3,+0.0
108 h,-2.4,+3.2,-1.1,-2.8,+0.9,-0.9,-2.0,+2.6,-1.6,-1.0,+0.1,-0.1,+0.1,-3.7,-0.3,-5.8,-1.8,-5.1,+0.8
120 h,-1.3,+2.2,-0.2,-0.1,-0.4,-1.3,-2.1,+4.0,-1.7,-1.8,-0.1,+0.1,+2.0,-2.0,-0.4,-4.8,-2.9,-5.7,+1.5


### 53986

In [16]:
heatmap_plot(data,'53986','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.7,-1.1,-1.6,-0.4,-1.4,-2.1,-1.7,-0.1,-0.9,-0.9,-0.6,-1.4,-1.2,-2.4,-1.1,-0.1,+1.6,-1.3,+2.7
36 h,-2.2,-1.0,-1.5,-0.8,-2.2,-2.5,-2.0,-0.1,-1.4,-0.8,-0.8,-1.2,-0.5,-1.4,-2.2,-0.2,+0.8,-1.3,+0.7
48 h,-3.1,+0.0,-2.1,-1.4,-0.8,-2.4,-2.3,-0.1,-1.1,-0.9,-1.0,-0.7,-3.4,-1.5,-0.5,-0.4,+1.8,-1.2,+0.8
60 h,-2.4,-1.3,-4.0,+0.1,-1.6,-2.2,-2.6,-0.3,-1.2,-0.9,-1.2,-1.4,-1.9,-2.6,-0.1,-1.7,+2.5,-1.5,+3.1
72 h,-2.7,-0.4,-2.9,-4.8,-1.3,-1.7,-2.3,-0.7,-1.0,-0.9,-1.3,-1.4,-0.8,-1.3,-0.6,-2.0,+2.9,-2.9,+3.4
84 h,-2.4,+0.7,-2.3,-3.2,+1.6,-3.5,-2.2,-0.9,-0.8,-1.5,-1.0,-1.8,-1.5,-1.0,-1.4,-4.4,+0.9,-1.1,+0.7
96 h,-2.8,-0.2,-3.0,-1.1,+0.2,-1.3,-2.0,-0.4,-0.9,-0.9,-1.4,-1.7,-1.0,-0.9,-0.7,-4.0,+0.1,-0.9,-0.2
108 h,-2.8,+2.0,-2.4,-3.2,+0.9,-1.2,-2.2,+2.6,-1.8,-1.1,-0.6,-1.3,-1.8,-4.3,-1.2,-5.1,-1.4,-5.6,+0.8
120 h,-2.0,+0.9,-1.5,-0.6,+0.2,-1.6,-2.4,+4.2,-1.8,-1.9,-0.8,-1.1,+1.7,-2.5,-0.8,-4.7,-2.2,-6.7,+1.1


### 53983

In [17]:
heatmap_plot(data,'53983','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-2.2,+1.4,+0.8,+1.2,-1.7,-1.9,-1.0,+0.0,-0.8,+0.2,+1.4,+0.9,+0.9,-0.6,+0.0,+0.1,+2.5,-0.7,+2.3
36 h,-2.2,+1.8,+0.5,+0.2,-2.7,-1.9,-1.6,+0.0,-0.8,+0.0,+1.3,+0.2,+1.8,-0.1,-0.4,-0.5,+4.2,+0.1,+0.1
48 h,-2.5,+2.6,+1.4,-2.3,-1.4,-1.7,-1.6,+0.2,-0.5,+0.1,+1.0,+1.6,+1.6,-0.7,+0.3,+0.1,+2.8,-0.7,+0.7
60 h,-2.6,+1.6,-1.2,+1.7,-1.5,-2.1,-1.8,-0.1,-0.8,+0.1,+0.8,+0.7,+0.9,-2.1,+0.5,-0.8,+3.5,-0.0,+2.8
72 h,-2.5,+1.7,-0.4,-3.0,-0.9,-1.7,-1.7,-1.1,-0.4,+0.1,+0.9,+0.6,+1.7,+0.4,+0.2,-2.5,+3.6,-0.8,+3.6
84 h,-2.4,+2.3,+0.1,-1.9,+1.5,-5.7,-1.7,-1.2,-0.4,-0.4,+0.9,+0.4,+1.1,+0.3,-1.2,-4.4,+3.5,-0.2,+2.0
96 h,-2.3,+3.1,+1.1,-1.0,+0.2,-1.8,-1.6,-0.0,-0.4,+0.0,+0.6,+0.4,+4.0,-0.3,+1.0,-3.5,+1.1,+0.1,+0.4
108 h,-3.2,+4.9,+1.1,-2.3,+0.7,-0.1,-1.9,+2.9,-0.9,+0.2,+1.5,+1.6,+3.1,-3.6,+1.2,-6.6,-2.8,-3.6,+0.3
120 h,-1.4,+4.4,+2.1,+0.3,-1.6,-1.2,-1.6,+4.3,-1.1,-0.9,+1.4,+1.6,+1.2,-1.5,+0.3,-4.2,-4.3,-4.1,+1.0


### 53985

In [18]:
heatmap_plot(data,'53985','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.0,-1.0,-1.4,-0.6,-1.0,-2.3,-2.7,-1.9,-1.9,-2.0,-1.3,-2.2,-1.6,-2.8,-1.1,-0.8,+1.3,-2.1,+1.7
36 h,-1.9,-1.0,-1.5,-0.5,-1.5,-2.3,-3.1,-1.6,-2.0,-2.0,-1.4,-1.0,-0.6,-2.5,-2.8,-1.0,-0.2,-1.6,+0.1
48 h,-2.5,-0.0,-2.3,-0.5,+0.1,-2.9,-3.5,-1.7,-1.7,-2.2,-1.7,-0.6,-2.6,-2.2,-0.7,-0.9,+1.2,-1.5,+0.2
60 h,-2.0,-0.8,-4.1,+0.6,-0.3,-2.3,-3.5,-1.9,-1.8,-2.1,-1.8,-1.3,-1.5,-2.8,-0.1,-2.8,+1.7,-1.7,+3.4
72 h,-2.2,-0.2,-2.5,-3.7,+0.0,-1.5,-3.3,-2.3,-1.5,-2.1,-1.9,-1.0,-0.2,-2.4,-0.7,-2.1,+2.2,-3.4,+4.1
84 h,-1.9,+0.2,-2.9,-2.9,+1.8,-3.2,-3.3,-2.5,-1.5,-2.6,-1.6,-1.3,-1.2,-1.2,-1.1,-5.6,-0.2,-2.0,+0.2
96 h,-2.0,-1.1,-3.2,-0.7,+0.5,-1.2,-3.2,-2.1,-1.9,-1.9,-2.0,-1.1,-0.7,-1.4,-0.4,-4.8,-1.4,-0.7,-0.4
108 h,-2.1,+1.4,-2.6,-2.7,+1.3,-1.4,-2.9,+0.9,-2.4,-2.1,-1.4,-1.3,-0.2,-3.4,-1.0,-5.8,-2.6,-6.2,+1.5
120 h,-1.5,+0.0,-2.1,-0.1,+1.0,-0.9,-3.0,+1.2,-2.5,-2.9,-1.9,-0.7,+3.1,-2.1,-0.4,-5.7,-2.6,-7.1,+1.5


### 53988

In [19]:
heatmap_plot(data,'53988','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.4,-1.9,-1.7,+1.1,-1.6,-1.8,-1.5,-0.7,-1.1,-1.1,-0.3,-0.2,+0.2,-2.3,-1.2,-0.4,+1.5,-0.9,+2.7
36 h,-1.9,-1.7,-2.1,+0.1,-2.8,-1.9,-1.8,-0.5,-1.2,-1.0,-0.6,+0.0,+0.8,-1.7,-1.7,-0.4,+0.4,-0.6,+0.8
48 h,-2.7,-0.9,-2.5,+0.4,-1.2,-1.7,-2.0,-0.7,-1.0,-1.1,-0.9,+0.4,-1.9,-1.4,-0.2,-0.9,+1.5,-0.8,+0.8
60 h,-1.9,-1.7,-4.5,+1.5,-1.7,-1.8,-2.3,-0.9,-1.2,-1.1,-0.9,-0.3,+0.2,-2.8,+0.2,-1.6,+2.4,-1.1,+2.7
72 h,-2.2,-1.0,-3.0,-3.1,-0.9,-1.0,-2.2,-1.3,-1.1,-1.1,-1.0,-0.2,+0.5,-3.2,-0.4,-2.1,+2.9,-2.8,+3.2
84 h,-2.3,+0.2,-3.1,-1.4,+1.3,-3.2,-2.1,-1.5,-0.6,-1.7,-0.7,-0.7,-0.0,-0.9,-1.3,-4.9,+1.9,-0.8,+0.9
96 h,-2.2,-0.5,-3.9,+0.2,-0.4,-0.5,-2.0,-1.1,-0.8,-1.0,-1.0,-0.6,+1.5,-1.2,-0.4,-4.6,-1.0,-0.2,-0.7
108 h,-2.2,+1.7,-3.9,-2.7,+1.1,-0.5,-2.0,+1.9,-1.6,-1.2,-0.4,-0.4,+0.6,-4.8,-1.0,-5.5,-1.7,-6.0,+1.4
120 h,-1.6,+0.0,-3.1,+0.9,+0.4,-1.1,-2.2,+3.4,-1.6,-1.9,-0.8,-0.1,+3.1,-2.3,-1.0,-4.7,-2.2,-6.8,+2.1


### 53989

In [20]:
heatmap_plot(data,'53989','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.5,+0.1,-1.0,+0.4,-1.4,-2.2,-0.8,+0.7,-0.7,-0.8,+0.4,+0.2,+0.1,-0.9,-0.7,-0.1,+2.5,-0.3,+3.4
36 h,-2.0,-0.1,-1.1,-0.9,-2.7,-2.4,-1.2,+0.9,-0.7,-0.9,+0.2,-0.3,+0.9,-0.4,-0.6,-0.1,+3.0,-0.3,+1.2
48 h,-2.3,+1.0,-1.4,-1.5,-1.1,-2.3,-1.2,+0.8,-0.7,-0.9,+0.1,+0.6,-0.5,-0.7,-0.2,-0.3,+2.9,-0.6,+1.2
60 h,-2.3,-0.5,-3.1,+1.2,-1.5,-2.1,-1.5,+0.6,-0.8,-0.9,-0.2,-0.2,-0.2,-1.9,+0.2,-0.9,+3.7,-0.5,+3.9
72 h,-2.3,+0.2,-2.0,-4.7,-0.8,-1.5,-1.3,+0.1,-0.6,-0.9,-0.2,-0.3,+0.9,-0.6,-0.5,-2.5,+3.9,-1.6,+4.0
84 h,-2.1,+1.2,-1.4,-2.7,+1.5,-4.2,-1.3,-0.1,-0.4,-1.3,-0.1,-0.8,+0.2,-0.1,-1.6,-4.6,+4.3,+0.1,+2.2
96 h,-2.5,+0.5,-1.8,-1.3,+0.3,-1.6,-1.1,+0.7,-0.3,-0.9,-0.4,-0.8,+1.7,-0.2,+0.2,-4.9,+0.6,-0.3,+0.3
108 h,-2.6,+3.2,-1.4,-3.2,+1.2,-1.3,-1.3,+3.8,-1.1,-0.9,+0.4,+0.2,+0.5,-3.7,-0.2,-6.2,-0.6,-5.0,+1.4
120 h,-1.6,+2.2,-0.4,+0.4,+0.3,-1.5,-1.5,+5.3,-1.2,-1.8,+0.4,+0.2,+2.2,-2.4,-0.4,-4.7,-2.7,-6.1,+2.5


### 53994

In [21]:
heatmap_plot(data,'53994','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.0,-0.0,-1.1,-0.7,-1.5,-2.5,-1.5,-0.3,-1.2,-1.4,-0.4,-0.7,-0.6,-2.1,-0.9,+0.3,+1.9,-0.6,+2.7
36 h,-1.9,-0.1,-1.1,-1.1,-1.9,-3.0,-2.0,-0.1,-1.7,-1.5,-0.6,-0.4,-0.5,-0.2,-2.9,+0.4,+1.2,-0.8,+0.4
48 h,-2.3,+1.3,-0.8,-2.2,-0.9,-2.7,-2.0,-0.1,-1.4,-1.6,-0.6,+0.4,-2.0,-0.8,-0.8,+0.4,+2.2,-0.7,+0.6
60 h,-1.7,-0.3,-3.0,-0.4,-1.8,-2.7,-2.3,-0.3,-1.5,-1.5,-0.9,-0.5,-1.4,-1.9,-0.0,-1.3,+2.4,-0.8,+3.5
72 h,-2.0,+0.8,-2.2,-5.4,-1.7,-2.4,-2.0,-0.8,-1.2,-1.5,-1.0,-0.4,-0.6,-0.2,-0.7,-1.3,+3.3,-2.1,+3.5
84 h,-1.7,+1.6,-1.5,-4.1,+1.7,-4.3,-1.9,-0.9,-1.1,-1.9,-0.8,-0.7,-0.8,-0.4,-1.5,-3.5,+0.2,-0.6,+0.3
96 h,-2.0,+0.3,-1.8,-1.1,+0.5,-1.8,-1.9,-0.4,-1.3,-1.5,-1.2,-0.8,-1.0,-0.1,-0.9,-3.2,+1.0,-0.2,+0.3
108 h,-2.1,+3.2,-1.1,-3.6,+0.8,-1.6,-2.2,+2.3,-2.1,-1.7,-0.3,-0.2,-1.2,-3.3,-1.1,-4.4,-1.3,-4.5,+0.2
120 h,-1.2,+1.9,-0.4,-1.2,-0.5,-2.0,-2.4,+4.3,-2.2,-2.6,-0.4,-0.1,+2.5,-1.7,-0.5,-4.5,-2.0,-5.6,+2.1


### 53997

In [22]:
heatmap_plot(data,'53997','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.6,+0.7,-0.8,-0.1,-1.4,-1.9,-1.0,-0.5,-0.8,-0.6,+0.7,-0.2,-1.3,-1.8,-0.7,-0.4,+1.6,-0.3,+2.2
36 h,-2.0,+1.2,-1.0,-0.8,-2.2,-2.1,-1.6,-0.5,-1.0,-0.6,+0.6,-0.7,-0.5,-0.8,-1.1,-0.7,+3.2,-0.5,+0.3
48 h,-2.2,+2.1,-0.5,-2.7,-1.2,-1.9,-1.4,-0.4,-0.8,-0.6,+0.4,+0.4,-0.8,-1.3,-0.3,-0.6,+2.8,-0.9,+0.3
60 h,-2.3,+0.7,-2.7,+0.8,-1.6,-2.1,-1.8,-0.6,-0.9,-0.6,+0.1,-0.5,-0.9,-2.5,+0.0,-1.7,+3.5,-0.7,+2.6
72 h,-2.1,+1.2,-2.1,-4.3,-1.0,-1.8,-1.7,-1.4,-0.7,-0.7,+0.1,-0.5,-0.1,-0.5,-0.5,-2.7,+3.6,-2.0,+2.9
84 h,-2.0,+1.8,-1.3,-3.2,+1.6,-4.6,-1.5,-1.6,-0.6,-1.0,+0.3,-0.8,-0.9,-0.5,-1.7,-4.7,+3.2,-0.0,+1.0
96 h,-2.4,+1.8,-1.5,-1.8,+0.6,-1.4,-1.5,-0.6,-0.5,-0.6,-0.1,-1.1,+1.0,-0.6,+0.0,-5.2,+0.7,-0.6,-0.1
108 h,-2.4,+4.3,-0.8,-3.2,+0.8,-0.9,-1.5,+2.6,-1.3,-0.7,+0.9,+0.2,-0.5,-4.2,-0.3,-6.5,-1.3,-5.3,-0.2
120 h,-1.3,+3.3,+0.0,-0.7,-0.7,-1.2,-1.6,+4.2,-1.4,-1.7,+0.7,+0.2,+0.5,-2.6,-0.5,-5.2,-3.1,-5.8,+1.0


### 53998

In [23]:
heatmap_plot(data,'53998','max_tmp_error', model)

日期,080920,081020,081120,081220,081320,081420,081520,081620,081720,081820,081920,082020,082120,082220,082320,082420,082520,082620,082720
时效,,,,,,,,,,,,,,,,,,,
24 h,-1.1,+2.0,+2.3,+1.5,-1.7,-1.4,-1.0,+0.3,-1.1,-0.4,+0.5,+0.1,-1.1,-0.0,+0.1,+0.3,+2.1,-0.6,+2.8
36 h,-1.3,+2.0,+1.4,+0.9,-1.7,-1.4,-1.5,+0.2,-1.3,-0.6,+0.3,-1.2,-0.4,+0.9,-1.5,-0.6,+3.9,+0.2,+1.0
48 h,-1.5,+2.2,+2.8,-0.9,-1.5,-1.3,-1.5,+0.7,-1.1,-0.5,+0.1,+0.3,-1.2,+0.2,+0.1,+0.1,+3.1,-1.1,+0.9
60 h,-1.4,+2.0,+0.3,+2.5,-1.8,-1.7,-1.9,+0.2,-1.2,-0.5,-0.0,-0.4,-1.8,-1.2,+0.7,-1.4,+3.8,-0.3,+3.4
72 h,-1.4,+1.6,+1.2,-2.6,-1.2,-1.1,-1.5,-0.9,-0.8,-0.5,+0.1,-0.2,-1.1,+1.2,+0.2,-2.2,+3.8,-0.7,+3.5
84 h,-2.6,+2.2,+1.2,-2.0,+1.9,-6.2,-1.5,-0.8,-0.9,-0.8,+0.0,-0.5,-1.2,+0.9,-0.9,-3.7,+3.3,+0.3,+1.6
96 h,-1.1,+3.1,+3.3,+1.4,+0.4,-1.9,-1.7,+0.3,-1.2,-0.5,-0.2,-0.5,+1.7,+0.8,+0.1,-3.0,+1.5,+0.8,+0.6
108 h,-2.1,+4.9,+2.5,-1.5,+0.8,+0.1,-2.0,+3.4,-1.5,-0.6,+0.7,+0.4,+0.3,-2.6,+1.3,-6.3,-2.6,-4.5,+0.8
120 h,-0.2,+4.8,+4.0,-0.1,-2.5,-1.1,-1.8,+4.9,-1.9,-1.0,+0.5,+0.4,+1.9,-0.7,+0.4,-4.5,-3.7,-3.3,+0.7


 ## <center> <font size=5> <b> 2m最低温度预报</b> </font> </center>

### 平均

In [24]:
heatmap_plot(data,'average','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,21.5,nan,nan,nan,nan,nan,nan,nan
36 h,21.1,22.3,nan,nan,nan,nan,nan,nan
48 h,22.4,22.2,nan,nan,nan,nan,nan,nan
60 h,21.8,20.9,19.0,nan,nan,nan,nan,nan
72 h,23.7,20.0,18.5,nan,nan,nan,nan,nan
84 h,23.3,20.6,19.4,19.6,nan,nan,nan,nan
96 h,25.1,21.0,20.4,20.4,nan,nan,nan,nan
108 h,23.8,21.5,20.1,20.4,20.6,nan,nan,nan
120 h,22.4,21.1,20.6,21.6,20.7,nan,nan,nan


### 53986

In [25]:
heatmap_plot(data,'53986','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,21.1,nan,nan,nan,nan,nan,nan,nan
36 h,20.6,22.8,nan,nan,nan,nan,nan,nan
48 h,22.2,22.3,nan,nan,nan,nan,nan,nan
60 h,21.5,20.3,19.3,nan,nan,nan,nan,nan
72 h,23.7,19.2,18.7,nan,nan,nan,nan,nan
84 h,23.6,20.1,20.1,19.7,nan,nan,nan,nan
96 h,25.1,20.6,20.4,20.0,nan,nan,nan,nan
108 h,24.0,21.4,20.5,20.2,20.1,nan,nan,nan
120 h,22.3,21.2,20.4,21.0,20.4,nan,nan,nan


### 53983

In [26]:
heatmap_plot(data,'53983','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,21.8,nan,nan,nan,nan,nan,nan,nan
36 h,21.7,22.3,nan,nan,nan,nan,nan,nan
48 h,22.9,22.4,nan,nan,nan,nan,nan,nan
60 h,22.2,21.7,18.6,nan,nan,nan,nan,nan
72 h,23.7,20.8,17.9,nan,nan,nan,nan,nan
84 h,23.8,21.4,19.7,19.3,nan,nan,nan,nan
96 h,25.3,20.9,20.1,21.4,nan,nan,nan,nan
108 h,24.1,21.4,19.9,20.8,20.9,nan,nan,nan
120 h,22.1,20.4,20.3,22.2,21.3,nan,nan,nan


### 53985

In [27]:
heatmap_plot(data,'53985','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,20.6,nan,nan,nan,nan,nan,nan,nan
36 h,20.5,22.7,nan,nan,nan,nan,nan,nan
48 h,21.8,22.1,nan,nan,nan,nan,nan,nan
60 h,20.9,21.8,20.1,nan,nan,nan,nan,nan
72 h,23.3,20.3,20.3,nan,nan,nan,nan,nan
84 h,23.2,21.5,20.0,19.6,nan,nan,nan,nan
96 h,24.8,21.8,20.9,21.3,nan,nan,nan,nan
108 h,23.8,22.4,20.2,21.0,20.7,nan,nan,nan
120 h,22.4,22.8,21.0,21.6,20.9,nan,nan,nan


### 53988

In [28]:
heatmap_plot(data,'53988','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,21.4,nan,nan,nan,nan,nan,nan,nan
36 h,20.6,21.7,nan,nan,nan,nan,nan,nan
48 h,22.3,21.9,nan,nan,nan,nan,nan,nan
60 h,21.6,19.6,18.9,nan,nan,nan,nan,nan
72 h,23.5,19.2,18.2,nan,nan,nan,nan,nan
84 h,23.1,20.1,18.7,20.5,nan,nan,nan,nan
96 h,25.2,20.9,20.9,20.1,nan,nan,nan,nan
108 h,23.8,22.4,20.0,20.9,19.8,nan,nan,nan
120 h,22.6,22.3,20.8,21.3,20.8,nan,nan,nan


### 53989

In [29]:
heatmap_plot(data,'53989','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,22.1,nan,nan,nan,nan,nan,nan,nan
36 h,21.4,21.5,nan,nan,nan,nan,nan,nan
48 h,22.3,21.9,nan,nan,nan,nan,nan,nan
60 h,21.7,20.1,18.5,nan,nan,nan,nan,nan
72 h,23.9,18.7,18.1,nan,nan,nan,nan,nan
84 h,23.6,20.3,19.4,19.6,nan,nan,nan,nan
96 h,25.1,20.6,20.2,19.5,nan,nan,nan,nan
108 h,23.5,21.4,19.4,19.8,20.4,nan,nan,nan
120 h,22.2,20.4,20.1,21.9,21.1,nan,nan,nan


### 53994

In [30]:
heatmap_plot(data,'53994','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,21.1,nan,nan,nan,nan,nan,nan,nan
36 h,20.7,23.2,nan,nan,nan,nan,nan,nan
48 h,22.5,22.1,nan,nan,nan,nan,nan,nan
60 h,21.9,21.3,19.2,nan,nan,nan,nan,nan
72 h,23.8,20.9,19.1,nan,nan,nan,nan,nan
84 h,23.0,20.7,19.5,19.4,nan,nan,nan,nan
96 h,24.9,20.7,20.3,20.6,nan,nan,nan,nan
108 h,23.7,21.0,21.0,19.9,21.4,nan,nan,nan
120 h,23.0,20.8,20.0,21.1,20.4,nan,nan,nan


### 53997

In [31]:
heatmap_plot(data,'53997','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,22.0,nan,nan,nan,nan,nan,nan,nan
36 h,21.3,22.6,nan,nan,nan,nan,nan,nan
48 h,22.2,23.1,nan,nan,nan,nan,nan,nan
60 h,22.2,20.8,18.7,nan,nan,nan,nan,nan
72 h,23.8,19.5,18.1,nan,nan,nan,nan,nan
84 h,23.2,20.8,19.4,19.3,nan,nan,nan,nan
96 h,25.2,20.5,20.0,20.6,nan,nan,nan,nan
108 h,23.5,21.3,19.5,20.5,21.4,nan,nan,nan
120 h,22.3,19.8,20.0,21.2,20.7,nan,nan,nan


### 53998

In [32]:
heatmap_plot(data,'53998','min_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,21.7,nan,nan,nan,nan,nan,nan,nan
36 h,21.8,21.7,nan,nan,nan,nan,nan,nan
48 h,22.7,21.7,nan,nan,nan,nan,nan,nan
60 h,22.1,21.6,18.7,nan,nan,nan,nan,nan
72 h,23.8,21.9,17.9,nan,nan,nan,nan,nan
84 h,23.2,20.3,18.4,19.3,nan,nan,nan,nan
96 h,24.9,21.8,20.1,20.0,nan,nan,nan,nan
108 h,23.9,20.7,19.9,19.8,20.1,nan,nan,nan
120 h,22.7,21.3,22.3,22.3,20.1,nan,nan,nan


## <center> <font size=5> <b> 2m最高温度预报</b> </font> </center>

### 平均

In [33]:
heatmap_plot(data,'average','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.3,nan,nan,nan,nan,nan,nan,nan
36 h,32.9,30.4,nan,nan,nan,nan,nan,nan
48 h,33.4,30.8,nan,nan,nan,nan,nan,nan
60 h,33.4,31.1,31.2,nan,nan,nan,nan,nan
72 h,33.8,32.3,31.6,nan,nan,nan,nan,nan
84 h,34.0,31.9,31.4,31.4,nan,nan,nan,nan
96 h,34.3,32.5,31.8,31.8,nan,nan,nan,nan
108 h,32.9,30.7,31.5,32.0,31.7,nan,nan,nan
120 h,33.4,31.4,31.8,31.9,31.9,nan,nan,nan


### 53986

In [34]:
heatmap_plot(data,'53986','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.3,nan,nan,nan,nan,nan,nan,nan
36 h,32.8,30.2,nan,nan,nan,nan,nan,nan
48 h,33.7,30.6,nan,nan,nan,nan,nan,nan
60 h,33.4,30.8,31.0,nan,nan,nan,nan,nan
72 h,33.7,32.2,31.4,nan,nan,nan,nan,nan
84 h,34.3,31.8,31.1,31.2,nan,nan,nan,nan
96 h,34.5,32.1,31.7,31.6,nan,nan,nan,nan
108 h,32.6,30.4,31.3,31.8,31.5,nan,nan,nan
120 h,33.2,31.3,31.7,31.6,31.8,nan,nan,nan


### 53983

In [35]:
heatmap_plot(data,'53983','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.2,nan,nan,nan,nan,nan,nan,nan
36 h,32.9,30.8,nan,nan,nan,nan,nan,nan
48 h,33.4,31.4,nan,nan,nan,nan,nan,nan
60 h,33.4,31.8,31.5,nan,nan,nan,nan,nan
72 h,33.6,32.7,32.0,nan,nan,nan,nan,nan
84 h,33.9,32.2,31.8,31.9,nan,nan,nan,nan
96 h,34.1,32.9,32.2,32.2,nan,nan,nan,nan
108 h,33.1,31.2,31.8,32.4,32.0,nan,nan,nan
120 h,33.5,31.6,32.1,32.3,32.4,nan,nan,nan


### 53985

In [36]:
heatmap_plot(data,'53985','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.1,nan,nan,nan,nan,nan,nan,nan
36 h,32.9,29.9,nan,nan,nan,nan,nan,nan
48 h,33.0,30.0,nan,nan,nan,nan,nan,nan
60 h,33.3,30.3,30.4,nan,nan,nan,nan,nan
72 h,34.0,31.8,30.9,nan,nan,nan,nan,nan
84 h,33.8,31.5,30.6,30.4,nan,nan,nan,nan
96 h,34.1,32.4,30.8,30.8,nan,nan,nan,nan
108 h,33.1,30.6,30.9,31.0,30.8,nan,nan,nan
120 h,33.9,31.7,31.1,31.0,31.1,nan,nan,nan


### 53988

In [37]:
heatmap_plot(data,'53988','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.5,nan,nan,nan,nan,nan,nan,nan
36 h,33.2,30.5,nan,nan,nan,nan,nan,nan
48 h,33.5,30.9,nan,nan,nan,nan,nan,nan
60 h,33.3,31.1,31.2,nan,nan,nan,nan,nan
72 h,34.1,32.5,31.7,nan,nan,nan,nan,nan
84 h,34.2,31.6,31.6,31.4,nan,nan,nan,nan
96 h,34.7,32.4,31.6,31.9,nan,nan,nan,nan
108 h,33.1,30.9,31.4,32.2,31.8,nan,nan,nan
120 h,33.7,31.5,31.7,31.7,32.0,nan,nan,nan


### 53989

In [38]:
heatmap_plot(data,'53989','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.4,nan,nan,nan,nan,nan,nan,nan
36 h,32.8,30.6,nan,nan,nan,nan,nan,nan
48 h,33.3,31.1,nan,nan,nan,nan,nan,nan
60 h,33.3,31.3,31.3,nan,nan,nan,nan,nan
72 h,33.6,32.6,31.7,nan,nan,nan,nan,nan
84 h,34.0,32.0,31.5,31.7,nan,nan,nan,nan
96 h,34.0,32.5,31.9,32.0,nan,nan,nan,nan
108 h,32.9,30.9,31.6,32.2,31.9,nan,nan,nan
120 h,33.4,31.6,31.9,31.9,32.0,nan,nan,nan


### 53994

In [39]:
heatmap_plot(data,'53994','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.1,nan,nan,nan,nan,nan,nan,nan
36 h,32.6,30.3,nan,nan,nan,nan,nan,nan
48 h,33.7,30.7,nan,nan,nan,nan,nan,nan
60 h,33.5,30.8,31.2,nan,nan,nan,nan,nan
72 h,33.8,32.2,31.5,nan,nan,nan,nan,nan
84 h,33.8,32.0,31.1,31.3,nan,nan,nan,nan
96 h,34.6,32.2,31.7,31.5,nan,nan,nan,nan
108 h,32.4,30.2,31.4,31.8,31.5,nan,nan,nan
120 h,33.0,31.3,31.6,31.7,31.8,nan,nan,nan


### 53997

In [40]:
heatmap_plot(data,'53997','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.4,nan,nan,nan,nan,nan,nan,nan
36 h,33.0,30.3,nan,nan,nan,nan,nan,nan
48 h,33.3,30.9,nan,nan,nan,nan,nan,nan
60 h,33.4,31.1,31.2,nan,nan,nan,nan,nan
72 h,33.7,32.4,31.6,nan,nan,nan,nan,nan
84 h,34.1,32.0,31.5,31.8,nan,nan,nan,nan
96 h,34.1,32.4,32.1,32.1,nan,nan,nan,nan
108 h,32.8,30.7,31.7,32.4,31.9,nan,nan,nan
120 h,33.2,31.3,32.0,32.3,32.3,nan,nan,nan


### 53998

In [41]:
heatmap_plot(data,'53998','max_tmp', model)

日期,082820,082920,083020,083120,090120,090220,090320,090420
时效,,,,,,,,
24 h,33.3,nan,nan,nan,nan,nan,nan,nan
36 h,33.4,30.6,nan,nan,nan,nan,nan,nan
48 h,33.5,31.1,nan,nan,nan,nan,nan,nan
60 h,33.5,31.5,31.6,nan,nan,nan,nan,nan
72 h,33.7,32.4,32.1,nan,nan,nan,nan,nan
84 h,33.9,32.1,32.0,31.7,nan,nan,nan,nan
96 h,34.1,32.7,32.5,31.9,nan,nan,nan,nan
108 h,32.9,30.7,32.0,32.3,31.9,nan,nan,nan
120 h,33.4,31.2,32.3,32.3,32.1,nan,nan,nan
